## Preprocessing Data



In [19]:
import pandas as pd 
import numpy as np 

raw = pd.read_csv("D:\\Programming\\Python_code\\PrinciplesOfDS_Course\\Labs\\Data\\insurance.csv")
print(raw.info())
print(raw.isna().sum()/raw.shape[0])

#separate out categorical features from numeric
categorical = raw.drop(columns=["age", "bmi", "charges", "children"])
print(categorical.info())

numeric = raw.drop(columns=["sex", "smoker", "region"])
print(numeric.info())

# convert categorical features to numeric 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

lenc = LabelEncoder()
categorical = categorical.apply(lenc.fit_transform) # convert all categoricals to numeric using .apply()
print(categorical.info())

# Inspect unique values: 
#print(categorical.apply(unique)) # surprised I cannot use this syntax... I guess because the DF lacks the method.. maybe only series can have unique used on them? weird. 

print('Col 1:',categorical.iloc[:,0].unique(), 'Col 2:',categorical.iloc[:,1].unique(),'Col 3:',categorical.iloc[:,2].unique())
#Col 1: [0 1] Col 2: [1 0] Col 3: [3 2 1 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None
age         0.0
sex         0.0
bmi         0.0
children    0.0
smoker      0.0
region      0.0
charges     0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sex     1338 non-null   object
 1   smoker  1338 non-null   object
 2   region  1338 non-null   object
dtypes: object(3)
memory usage: 31.5+ KB
None
<class 'pandas.c

In [33]:
# Recombining DFs, going to test the model on both non normalized binary cols as well as normalized binary cols. 

#preProc = pd.concat([numeric,categorical], ignore_index=True, sort=False) # .append is deprecated.
#print(preProc.info()) # important to note here, is whether or not any values would be shifted around, corrupting the data by mixing and matching values from different observations
# so using concat actually added the new columns as new rows and didn't attmept to bind them together col by col.  

preProc = numeric.join(categorical) # .join() works for just smashing two DFs together by index. 
print("Join version, uses indices as the key to join the two DFs from: \n", preProc.info())
# not sure why, but the info() table prints before everything else.. which is honestly confusing. Will have to remember this abnormal behavior. 

x_mean_pd = preProc.mean() # I want to see what difference comes of using the DF method for mean vs a numpy method. 
x_mean = np.mean(preProc, axis=0)
x_std = np.std(preProc)

print("\nMethod differences. pandas mean output and type:\n", x_mean_pd, type(x_mean_pd), "\n\nnumpy mean and output type:\n", x_mean, type(x_mean))
# all the same. So it is safe to not use the numpy method I suppose which is nice as the syntax is cleaner for the pandas version

X_allNorm = (preProc-x_mean)/x_std
print(X_allNorm)

# creating non-normalized binary feature matrix
preProc_2 = preProc.copy()
preProc_2['sex'] = categorical['sex']
preProc_2['smoker'] = categorical['smoker']

print(preProc.head())
print(preProc_2.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   bmi       1338 non-null   float64
 2   children  1338 non-null   int64  
 3   charges   1338 non-null   float64
 4   sex       1338 non-null   int32  
 5   smoker    1338 non-null   int32  
 6   region    1338 non-null   int32  
dtypes: float64(2), int32(3), int64(2)
memory usage: 57.6 KB
Join version, uses indices as the key to join the two DFs from: 
 <class 'pandas.core.frame.DataFrame'> 
 None (1338, 7)

Method differences. pandas mean output and type:
 age            39.207025
bmi            30.663397
children        1.094918
charges     13270.422265
sex             0.505232
smoker          0.204783
region          1.515695
dtype: float64 <class 'pandas.core.series.Series'> 

numpy mean and output type:
 age            39.207025
bmi            30.663397
chi